````SQL

CREATE DATABASE OCTOBER0120251;
USE OCTOBER0120251;

CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name VARCHAR(50),
    city VARCHAR(50),
    signup_date DATE
);

INSERT INTO Customers (customer_id, name, city, signup_date) VALUES
(1, 'Alice', 'New York', '2025-01-10'),
(2, 'Bob', 'Los Angeles', '2025-02-20'),
(3, 'Charlie', 'Chicago', '2025-03-15'),
(4, 'David', 'Houston', '2025-04-05'),
(5, 'Eva', 'New York', '2025-05-12');

CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    product_id INT,
    order_date DATE,
    amount DECIMAL(10,2),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

INSERT INTO Orders (order_id, customer_id, product_id, order_date, amount) VALUES
(101, 1, 201, '2025-06-01', 250.50),
(102, 2, 202, '2025-06-03', 120.00),
(103, 1, 203, '2025-06-07', 450.00),
(104, 3, 201, '2025-06-10', 300.00),
(105, 5, 204, '2025-06-12', 150.75);

CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(50),
    category VARCHAR(50),
    price DECIMAL(10,2)
);

INSERT INTO Products (product_id, product_name, category, price) VALUES
(201, 'Laptop', 'Electronics', 1000.00),
(202, 'Headphones', 'Electronics', 150.00),
(203, 'Desk Chair', 'Furniture', 200.00),
(204, 'Notebook', 'Stationery', 10.00);

CREATE TABLE Employees (
    emp_id INT PRIMARY KEY,
    name VARCHAR(50),
    manager_id INT
);

INSERT INTO Employees (emp_id, name, manager_id) VALUES
(1, 'John', NULL),
(2, 'Sara', 1),
(3, 'Mike', 1),
(4, 'Emma', 2),
(5, 'Ryan', 3);

SELECT * FROM Customers;
SELECT * FROM Orders;
SELECT * FROM Products;
SELECT * FROM Employees;

-- 1. Question 1: Combined Report
-- Get a list of all orders, showing: Customer name, City, Product name, Category, Order amount
-- Total price (use CASE to apply 10% discount if amount > 300, else no discount)
SELECT 
    C.name AS Customer, 
    C.City AS City, 
    P.product_name AS Product, 
    P.category AS Category, 
    O.amount AS Amount,
    CASE 
        WHEN O.amount > 300 THEN O.amount - (0.1 * O.amount)
        ELSE O.amount
    END AS total_price
FROM Orders O
JOIN Customers C ON O.customer_id = C.customer_id
JOIN Products P ON O.product_id = P.product_id;

-- 2. Find all customers who never placed an order, and show their name and city.
Select C.name AS Customer, C.City AS City, O.*
From Customers C LEFT JOIN Orders O
ON C.customer_id = O.customer_id
Where O.customer_id IS NULL;

-- 3. List all employees along with their manager's name. If no manager, display "No Manager".
SELECT 
    E.name AS Employee,
    CASE 
        WHEN M.name IS NULL THEN 'No Manager'
        ELSE M.name
    END AS Manager
FROM Employees E
LEFT JOIN Employees M ON E.manager_id = M.emp_id;

-- 4. Show a report that contains:
-- Product category, Total number of orders for that category
-- Average order amount, Number of distinct customers who ordered that category
SELECT 
    P.category AS Category,
    COUNT(O.order_id) AS Total_Orders,         
    AVG(O.amount) AS Average_Amount,           
    COUNT(DISTINCT O.customer_id) AS Distinct_Customers
FROM Products P
JOIN Orders O ON P.product_id = O.product_id
GROUP BY P.category;

-- 5. Get a list showing: City, Number of orders from that city, Total order amount, Average order amount
-- Include cities even if there are no orders from them.
Select C.City AS City, Count(O.order_id) AS Number_Orders, SUM(O.amount) AS Total_Amount,
AVG(o.amount) AS Average_Amount 
FROM Customers C LEFT JOIN Orders O
ON C.customer_id = O.customer_id
Group BY C.City;

-- 6. Create a report that lists all customer names from:
-- Customers table
-- Employees table (consider employees as potential customers)
Select name AS Customer From Customers
UNION ALL
Select name From Employees;

-- 7. Generate a list of all possible customer-product combinations, 
-- and show the expected total price if each customer buys one of each product.
SELECT 
    C.customer_id,
    C.name AS Customer,
    SUM(P.price) AS Total_Expected_Price
FROM Customers C
CROSS JOIN Products P
GROUP BY C.customer_id, C.name;

-- 8. List each customer and their most recent order (if any). Show: customer name, order id, product name, and order date.
SELECT 
    T.name AS Customer,
    T.order_id,
    P.product_name,
    T.order_date
FROM (
    SELECT 
        C.customer_id,
        C.name,
        O.order_id,
        O.product_id,
        O.order_date,
        DENSE_RANK() OVER (PARTITION BY C.customer_id ORDER BY O.order_date DESC) AS Ranking
    FROM Customers C
    LEFT JOIN Orders O ON C.customer_id = O.customer_id
) T
LEFT JOIN Products P ON T.product_id = P.product_id
WHERE T.Ranking = 1;
